In [87]:
# Importando os módulos:
import pandas as pd
import folium
from folium import plugins, Map
import matplotlib.pyplot as plt
import numpy as np
from API_google import reverse_geocode
import seaborn as sns
import csv 
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [73]:
# Nova data frame com os dados tratados (rodar todas as vezes)
df= pd.read_csv(r"dataset_consumo_2.csv", delimiter=';', encoding='ANSI')

In [74]:
df['LATITUDE'] = df['LATITUDE'].str.replace(',','.')
df['LONGITUDE'] = df['LONGITUDE'].str.replace(',','.')

In [75]:
df

,ID_USUARIO,DATA_NASCIMENTO,RENDA,RENDA_FIXA,LATITUDE,LONGITUDE,TIPO_PESSOA,CIDADE,ESTADO,LOGRADOURO,ANO,IDADE,FAIXA ETARIA,CONSUMO
0,d6c0cc1d-eb61-486a-b5c3-50725ef44d61,30/01/1981 00:00,"DE R$ 1.900,00 A R$ 3.800,00","3.104,00",-4.84758,-37.78439,CPF,Jaguaruana,Ceará,street_address,1981.0,40.0,30 a 40 anos,SERVICO
1,da0067e7-aa6b-4355-b373-ca17f6ac3b44,28/01/1978 00:00,"DE R$ 3.800,00 A R$ 7.600,00","5.703,00",-26.99011814,-48.63716523,CPF,Balneário Camboriú,Santa Catarina,street_address,1978.0,43.0,40 a 50 anos,FARMACIAS
2,2d729b8a-d53c-48e7-8bf5-33b021980c04,09/12/1954 00:00,"DE R$ 3.800,00 A R$ 7.600,00","3.901,00",-5.053671016,-42.81315203,CPF,Teresina,Piauí,street_address,1954.0,67.0,60 a 70 anos,SERVICO
3,62c0f1e6-36f2-4205-a134-232a58a6f8a5,28/08/1975 00:00,"DE R$ 788,00 A R$ 1.140,00","1.086,00",-15.059042,-48.163372,CNPJ,Mimoso de Goiás,Goiás,establishment,1975.0,46.0,40 a 50 anos,M.O.T.O.
4,0cef9719-0aa3-49a2-adea-9c09c04de071,17/06/1957 00:00,"DE R$ 788,00 A R$ 1.140,00",836,-17.694658,-42.52228,CPF,Capelinha,Minas Gerais,street_address,1957.0,64.0,60 a 70 anos,M.O.T.O.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,f2261fce-83c5-4f64-b145-85f46bcf18b1,08/10/1979 00:00,"DE R$ 1.900,00 A R$ 3.800,00","2.776,00",-11.60937194,-47.04750645,CPF,Porto Alegre do Tocantins,Tocantins,route,1979.0,42.0,40 a 50 anos,VAREJO
9996,bdaa7802-4650-44a4-93f7-952b72ec17a0,30/04/1980 00:00,"DE R$ 3.800,00 A R$ 7.600,00","4.099,00",-15.65049668,-47.79332059,CPF,Brasília,Distrito Federal,street_address,1980.0,41.0,40 a 50 anos,SERVICO
9997,e7b37cd5-714a-44d8-ad7f-4ed3aa6b0695,26/05/1976 00:00,"DE R$ 788,00 A R$ 1.140,00","1.052,00",-8.330678649,-36.13689703,CPF,São Caetano,Pernambuco,street_address,1976.0,45.0,40 a 50 anos,SERVICO
9998,167dbd7f-7ccd-4aad-a6b9-c3fb9d56c9cd,06/11/1985 00:00,"DE R$ 788,00 A R$ 1.140,00",958,NaN,NaN,CNPJ,NaN,NaN,NaN,1985.0,36.0,30 a 40 anos,ARTIGOS ELETRO


In [76]:
#Transformar o tipo das colunas para Categoria
df['RENDA'] = pd.Categorical(df['RENDA'])
df['CONSUMO'] = pd.Categorical(df['CONSUMO'])
df['IDADE'] = pd.Categorical(df['IDADE'])
df['LATITUDE'] = pd.Categorical(df['LATITUDE'])
df['LONGITUDE'] = pd.Categorical(df['LONGITUDE'])

In [77]:
#Codificando os dados categóricos
df['RENDA'] = df['RENDA'].cat.codes
df['CONSUMO'] = df['CONSUMO'].cat.codes
df['IDADE'] = df['IDADE'].cat.codes
df['LATITUDE'] = df['LATITUDE'].cat.codes
df['LONGITUDE'] = df['LONGITUDE'].cat.codes

In [78]:
df_cluster=df[['CONSUMO','RENDA','IDADE','LATITUDE','LONGITUDE']]

In [79]:
#Normalização com média 0 e desvio 1
normaliza = MinMaxScaler()

In [80]:
#Aplica a normalização
X_std = normaliza.fit_transform(df_cluster)

In [81]:
#transformando os dados normalizados em um dataframe
df_grupc_norma = pd.DataFrame(X_std, columns=['CONSUMO','RENDA','IDADE','LATITUDE','LONGITUDE'])

In [82]:
#Agora todos os valores estão entre 0 e 1
df_grupc_norma.head(2)

,CONSUMO,RENDA,IDADE,LATITUDE,LONGITUDE
0,0.809524,0.000000,0.219780,0.872531,0.074183
1,0.238095,0.333333,0.252747,0.667451,0.565953


In [83]:
#Realizando a aplicação do Kmeans
km = KMeans(n_clusters=4)
clusters = km.fit_predict(df_grupc_norma)
df_grupc_norma['CLUSTER'] = clusters
df_grupc_norma.head()

,CONSUMO,RENDA,IDADE,LATITUDE,LONGITUDE,CLUSTER
0,0.809524,0.000000,0.219780,0.872531,0.074183,2
1,0.238095,0.333333,0.252747,0.667451,0.565953,1
2,0.809524,0.333333,0.516484,0.875588,0.228192,2
3,0.523810,0.666667,0.285714,0.115122,0.531272,0
4,0.523810,0.666667,0.483516,0.187441,0.215495,0


In [84]:
cluster_0 = df_grupc_norma[df_grupc_norma['CLUSTER'] == 0]
cluster_1 = df_grupc_norma[df_grupc_norma['CLUSTER'] == 1]
cluster_2 = df_grupc_norma[df_grupc_norma['CLUSTER'] == 2]
cluster_3 = df_grupc_norma[df_grupc_norma['CLUSTER'] == 3]

In [85]:
cluster_1

,CONSUMO,RENDA,IDADE,LATITUDE,LONGITUDE,CLUSTER
1,0.238095,0.333333,0.252747,0.667451,0.565953,1
7,0.523810,0.000000,0.219780,0.917921,0.533153,1
11,0.523810,0.333333,0.758242,0.696731,0.851869,1
13,0.095238,0.000000,0.153846,0.873942,0.543851,1
15,0.809524,0.000000,0.406593,0.845484,0.872678,1
...,...,...,...,...,...,...
9981,0.523810,0.000000,0.395604,0.662982,0.585116,1
9987,0.095238,0.000000,0.263736,0.747295,0.879850,1
9989,0.952381,0.000000,0.263736,0.572907,0.446156,1
9992,0.809524,0.000000,0.252747,0.742827,0.742182,1


In [89]:
km.cluster_centers_

array([[0.74794085, 0.61196031, 0.4354274 , 0.4340126 , 0.58479075],
       [0.73451764, 0.05148964, 0.32110474, 0.57266563, 0.72895995],
       [0.74296681, 0.39120165, 0.34676218, 0.91204264, 0.10056175],
       [0.7400234 , 0.22515418, 0.34462898, 0.15018182, 0.16501763]])